In [1]:
import sys
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Adjust the path to point to your scripts folder relative to the notebook
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
from scipy import stats
from scripts.data_loader import load_data
import pandas as pd

In [2]:
df = load_data('/Users/elbethelzewdie/Documents/insurance-risk-analytics-week3/insurance-risk-analytics-week3/data/MachineLearningRating_v3.txt')

/Users/elbethelzewdie/Documents/insurance-risk-analytics-week3/insurance-risk-analytics-week3/scripts/data_loader.py:14: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path, sep=delimiter)


## Handling missing values before applying A/B Hypothesis Testing

In [3]:
print(df.isnull().sum())

UnderwrittenCoverID               0
PolicyID                          0
TransactionMonth                  0
IsVATRegistered                   0
Citizenship                       0
LegalType                         0
Title                             0
Language                          0
Bank                         145961
AccountType                   40232
MaritalStatus                  8259
Gender                         9536
Country                           0
Province                          0
PostalCode                        0
MainCrestaZone                    0
SubCrestaZone                     0
ItemType                          0
mmcode                          552
VehicleType                     552
RegistrationYear                  0
make                            552
Model                           552
Cylinders                       552
cubiccapacity                   552
kilowatts                       552
bodytype                        552
NumberOfDoors               

In [4]:
cols_to_drop = ['WrittenOff', 'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet']
df.drop(columns=cols_to_drop, inplace=True)

In [5]:
# 2. Drop rows where all vehicle-related columns are missing
vehicle_cols = ['mmcode', 'VehicleType', 'make', 'Model', 
                'Cylinders', 'cubiccapacity', 'kilowatts', 
                'bodytype', 'NumberOfDoors', 'VehicleIntroDate']

# Identify rows where all these columns are missing
rows_missing_all = df[vehicle_cols].isnull().all(axis=1)

# Drop these rows
df = df[~rows_missing_all]

In [10]:
df = df.dropna(subset=['CapitalOutstanding', 'CustomValueEstimate'])

In [6]:
# Fill Bank and AccountType with "Unknown"
cols_unknown = ['Bank', 'AccountType']
for col in cols_unknown:
    df[col] = df[col].fillna('Unknown')

# Fill Gender and MaritalStatus with mode
cols_mode = ['Gender', 'MaritalStatus']
for col in cols_mode:
    mode_value = df[col].mode()[0]
    df[col] = df[col].fillna(mode_value)

In [7]:
df['VehicleIntroDate'] = pd.to_datetime(df['VehicleIntroDate'], errors='coerce')
df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'])

df['vehicle_age_days'] = (df['TransactionMonth'] - df['VehicleIntroDate']).dt.days
df['vehicle_age_months'] = df['vehicle_age_days'] / 30.44

/var/folders/7h/pt6smvdj6k993z68shptnrdc0000gn/T/ipykernel_33145/1569193360.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['VehicleIntroDate'] = pd.to_datetime(df['VehicleIntroDate'], errors='coerce')


In [8]:
# Impute NewVehicle
df.loc[df['NewVehicle'].isnull() & (df['vehicle_age_months'] <= 6), 'NewVehicle'] = 'Less than 6 months'
df.loc[df['NewVehicle'].isnull() & (df['vehicle_age_months'] > 6), 'NewVehicle'] = 'More than 6 months'

In [11]:
print(df.isnull().sum())

UnderwrittenCoverID         0
PolicyID                    0
TransactionMonth            0
IsVATRegistered             0
Citizenship                 0
LegalType                   0
Title                       0
Language                    0
Bank                        0
AccountType                 0
MaritalStatus               0
Gender                      0
Country                     0
Province                    0
PostalCode                  0
MainCrestaZone              0
SubCrestaZone               0
ItemType                    0
mmcode                      0
VehicleType                 0
RegistrationYear            0
make                        0
Model                       0
Cylinders                   0
cubiccapacity               0
kilowatts                   0
bodytype                    0
NumberOfDoors               0
VehicleIntroDate            0
CustomValueEstimate         0
AlarmImmobiliser            0
TrackingDevice              0
CapitalOutstanding          0
NewVehicle

In [12]:
df['HasClaim'] = df['TotalClaims'] > 0

In [15]:
df.to_csv('/Users/elbethelzewdie/Documents/insurance-risk-analytics-week3/insurance-risk-analytics-week3/data/processed_insurance_data.csv', index=False)


Chi-squared tests are based on the so-called chi-squared statistic. You calculate the chi-squared statistic with the following formula:

$$
\chi^2 = \sum_{i=1}^{n} \frac{(O_i - E_i)^2}{E_i}
$$ 

In the formula, observed is the actual observed count for each category and expected is the expected count based on the distribution of the population for the corresponding category. Let's calculate the chi-squared statistic for our data to illustrate:

T-test

A t-test compares the means of two groups to see if the difference between them is statistically significant. It uses sample means and sample standard deviations to calculate a t-value and p-value.

$$
t = \frac{\bar{X}_1 - \bar{X}_2}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}
$$



Z-test

A z-test checks whether a sample mean differs significantly from a known population mean when the sample size is large or the population standard deviation is known. It uses the z-score to measure how far the sample mean is from the population mean in standard deviation units. 

$$
z = \frac{\bar{X} - \mu}{\sigma / \sqrt{n}}
$$

ANOVA

ANOVA compares the means of three or more groups to determine whether at least one group is significantly different from the others. It does this by analyzing how much variation exists between groups compared to within groups.

$$
F = \frac{MSB}{MSW}
$$

where

$$
MSB = \frac{\sum_i n_i (\bar{X}_i - \bar{X}_\text{overall})^2}{k - 1}
\quad\text{and}\quad
MSW = \frac{\sum_{i,j} (X_{ij} - \bar{X}_i)^2}{N - k}
$$


## ✅H0: No risk differences across provinces

Alternative Hypothesis (H₁):

- H₁: There are risk differences across provinces.

(Meaning: at least one province has a different risk level.)

#### Claim Frequency Across Provinces

In [19]:
# Chi-square test for independence
from scipy.stats import chi2_contingency

# Create contingency table: provinces × claim status
contingency = pd.crosstab(df['Province'], df['HasClaim'])

# Perform the chi-square test
chi2, p_freq, dof, expected = chi2_contingency(contingency)

print(f"Claim Frequency across provinces:")
print(f"group_col: {'Province'}")
print(f"Chi-square statistic: {chi2:.2f}")
print(f"P-value: {p_freq:.6f}")
print(f"Degrees of freedom: {dof}")
print(f"contingency table:\n{contingency}")

# If p < 0.05, reject H₀ - there ARE differences
if p_freq < 0.05:
    print("REJECT H₀: There ARE significant differences in claim frequency across provinces")
    
    # Identify which provinces differ most
    province_rates = df.groupby('Province')['HasClaim'].mean().sort_values()
    print("\nClaim frequencies by province:")
    print(province_rates)
else:
    print("FAIL TO REJECT H₀: No significant differences in claim frequency")

Claim Frequency across provinces:
group_col: Province
Chi-square statistic: 26.55
P-value: 0.000846
Degrees of freedom: 8
contingency table:
HasClaim       False  True 
Province                   
Eastern Cape    7395     15
Free State      1856      4
Gauteng        89794    323
KwaZulu-Natal  53103    154
Limpopo         7033     21
Mpumalanga      6934     25
North West     24839     53
Northern Cape    599      1
Western Cape   28244     63
REJECT H₀: There ARE significant differences in claim frequency across provinces

Claim frequencies by province:
Province
Northern Cape    0.001667
Eastern Cape     0.002024
North West       0.002129
Free State       0.002151
Western Cape     0.002226
KwaZulu-Natal    0.002892
Limpopo          0.002977
Gauteng          0.003584
Mpumalanga       0.003592
Name: HasClaim, dtype: float64


My analysis reveals statistically significant differences in claim frequency across South African provinces (p = 0.000846). While all provinces maintain relatively low claim rates (< 0.4%), there is a 2.2x variation between the lowest-risk province (Northern Cape at 0.17%) and the highest-risk province (Mpumalanga at 0.36%). This finding provides a data-driven foundation for geographic segmentation in our pricing strategy.

#### Risk Clusters Identified
##### Low-Risk Tier (0.17-0.22%):

Northern Cape, Eastern Cape, North West, Free State, Western Cape

Business implication: These provinces demonstrate superior risk profiles and may warrant preferential pricing

##### Medium-Risk Tier (0.29-0.30%):

KwaZulu-Natal, Limpopo

Business implication: Standard pricing approach appropriate

##### High-Risk Tier (0.36%):

Gauteng, Mpumalanga

Business implication: These provinces show significantly higher claim frequencies requiring pricing adjustments

#### Claim Severity Across Provinces


In [23]:
from scipy.stats import f_oneway

# Optional filtering: only customers who have at least 1 claim
condition_col = "HasClaim"
condition_value = 1

df_filtered = df[df[condition_col] == condition_value]

group_col = "Province"
value_col = "TotalClaims"

# Keep only relevant columns
df_filtered = df_filtered[[group_col, value_col]].dropna()

# Build groups
groups = []
for _, group in df_filtered.groupby(group_col):
    values = group[value_col].values
    if len(values) > 1:  # only include groups with >=2 entries
        groups.append(values)

# Count number of valid groups
group_count = len(groups)

# Check minimum requirement
if group_count < 2:
    print("Not enough groups to run ANOVA")
    print("Number of groups with sufficient data:", group_count)
else:
    f_stat, p_value = f_oneway(*groups)
    print("ANOVA Results for TotalClaims Across Provinces:")
    print("F-statistic:", f_stat)
    print("P-value:", p_value)
    print("Number of groups with sufficient data:", group_count)


ANOVA Results for TotalClaims Across Provinces:
F-statistic: 1.6641703569674584
P-value: 0.11486184525051764
Number of groups with sufficient data: 8


The ANOVA test for claim severity (TotalClaims amount) across provinces shows no statistically significant differences (p = 0.115). While there are visible variations in average claim amounts between provinces, these differences are not statistically reliable and could be due to random variation rather than true underlying risk patterns. This finding has important implications for our segmentation strategy.

- There is an 11.5% probability that we would observe these claim amount differences purely by chance, even if all provinces had identical true claim severity

- This exceeds our 5% significance threshold, meaning the evidence is not strong enough to conclude real differences exist

- The F-statistic of 1.66 is relatively low, indicating that between-province variation is not substantially greater than within-province variation

#### Marigin Across provinces

In [27]:

group_col = "Province"  # change this to the column you want to group by

# ------------------------------
# 1. Ensure Margin column exists
# ------------------------------
if 'Margin' not in df.columns:
    # Here Margin = TotalPremium - TotalClaims
    df['Margin'] = df['TotalPremium'] - df['TotalClaims']

# ------------------------------
# 2. Keep only relevant columns and drop NA
# ------------------------------
df_filtered = df[[group_col, 'Margin']].dropna()

# ------------------------------
# 3. Build groups for ANOVA
# ------------------------------
groups = []

for _, group in df_filtered.groupby(group_col):
    values = group['Margin'].values
    if len(values) > 1:  # only include groups with at least 2 entries
        groups.append(values)

# Count the number of valid groups
group_count = len(groups)

# ------------------------------
# 4. Check if enough groups to run ANOVA
# ------------------------------
if group_count < 2:
    print("Not enough data to perform ANOVA")
    print("Number of groups with sufficient data:", group_count)
else:
    # ------------------------------
    # 5. Run ANOVA
    # ------------------------------
    f_stat, p_val = f_oneway(*groups)

    print("ANOVA Results for Margin Across Groups:")
    print("Group Column:", group_col)
    print("Value Column: Margin")
    print("F-statistic:", f_stat)
    print("P-value:", p_val)
    print("Number of groups with sufficient data:", group_count)


ANOVA Results for Margin Across Groups:
Group Column: Province
Value Column: Margin
F-statistic: 1.277818432950938
P-value: 0.24975877187524445
Number of groups with sufficient data: 9


The ANOVA analysis reveals no statistically significant differences in profitability (Margin = Premium - Claims) across South African provinces (p = 0.250). Despite varying claim frequencies previously identified, overall profit margins remain statistically consistent nationwide. This critical finding suggests that our current pricing strategy effectively balances risk across provinces, maintaining uniform profitability despite differing risk profiles.

## ✅H0: No risk differences between zip codes

Alternative Hypothesis (H₁):

- H₁: There are risk differences between zip codes.

(Meaning: at least one zip code has a different risk level.)

#### Claim Frequency Across Zip Codes

In [25]:
# Create contingency table: provinces × claim status
contingency = pd.crosstab(df['PostalCode'], df['HasClaim'])


# Perform the chi-square test
chi2, p_freq, dof, expected = chi2_contingency(contingency)

print(f"Claim Frequency across provinces:")
print(f"group_col: {'Province'}")
print(f"Chi-square statistic: {chi2:.2f}")
print(f"P-value: {p_freq:.6f}")
print(f"Degrees of freedom: {dof}")
print(f"contingency table:\n{contingency}")

# If p < 0.05, reject H₀ - there ARE differences
if p_freq < 0.05:
    print("REJECT H₀: There ARE significant differences in claim frequency across provinces")
    
    # Identify which provinces differ most
    province_rates = df.groupby('PostalCode')['HasClaim'].mean().sort_values()
    print("\nClaim frequencies by province:")
    print(province_rates)
else:
    print("FAIL TO REJECT H₀: No significant differences in claim frequency")

Claim Frequency across provinces:
group_col: Province
Chi-square statistic: 689.24
P-value: 0.000000
Degrees of freedom: 497
contingency table:
HasClaim    False  True 
PostalCode              
1             336      0
2             270      0
8             247      0
17            160      0
29            300      0
...           ...    ...
9745           45      0
9750           60      0
9762            1      0
9781          212      1
9869           30      0

[498 rows x 2 columns]
REJECT H₀: There ARE significant differences in claim frequency across provinces

Claim frequencies by province:
PostalCode
1       0.000000
4008    0.000000
3974    0.000000
3973    0.000000
3965    0.000000
          ...   
1665    0.024390
4027    0.024793
7764    0.027174
2920    0.054545
466     0.055556
Name: HasClaim, Length: 498, dtype: float64


My analysis reveals extremely significant differences in claim frequency across postal codes (p ≈ 0.000000, Chi-square = 689.24). This represents the strongest statistical evidence we've found in all segmentation analyses. The risk variation is dramatic, ranging from 0% claim frequency in many areas to 5.56% in the highest-risk postal codes. This finding demands immediate strategic attention for pricing and underwriting.

#### Claim Severity Across Zip Codes

In [31]:
df_claims = df[df['HasClaim'] == True]

# Group claim severity by PostalCode
zip_groups = [
    group['TotalClaims'].dropna().values
    for name, group in df_claims.groupby('PostalCode')  # Replace with 'ZipCode' if that's your column name
]

# Run ANOVA
f_stat, p_severity_zip = f_oneway(*zip_groups)
print(f"Claim Severity ANOVA by ZipCode: F = {f_stat}, p-value = {p_severity_zip}")

Claim Severity ANOVA by ZipCode: F = 1.5622613436465371, p-value = 0.00010234117432428901


The ANOVA analysis reveals highly statistically significant differences in claim severity across postal codes (F = 1.562, p = 0.000102). Combined with our previous finding of extremely significant claim frequency differences (p ≈ 0.000000), this confirms that postal code affects BOTH claim likelihood AND claim costs. This dual impact makes postal code our most powerful segmentation variable, demanding immediate integration into pricing models.

## ✅H0: No significant margin difference between zip codes


Alternative Hypothesis (H₁):

- H₁: There is a significant margin (profit) difference between zip codes.

(Meaning: profit is not equal across all zip codes.)

In [32]:

group_col = "PostalCode"  # change this to the column you want to group by


# ------------------------------
# 1. Ensure Margin column exists
# ------------------------------
if 'Margin' not in df.columns:
    # Here Margin = TotalPremium - TotalClaims
    df['Margin'] = df['TotalPremium'] - df['TotalClaims']

# ------------------------------
# 2. Keep only relevant columns and drop NA
# ------------------------------
df_filtered = df[[group_col, 'Margin']].dropna()

# ------------------------------
# 3. Build groups for ANOVA
# ------------------------------
groups = []

for _, group in df_filtered.groupby(group_col):
    values = group['Margin'].values
    if len(values) > 1:  # only include groups with at least 2 entries
        groups.append(values)

# Count the number of valid groups
group_count = len(groups)

# ------------------------------
# 4. Check if enough groups to run ANOVA
# ------------------------------
if group_count < 2:
    print("Not enough data to perform ANOVA")
    print("Number of groups with sufficient data:", group_count)
else:
    # ------------------------------
    # 5. Run ANOVA
    # ------------------------------
    f_stat, p_val = f_oneway(*groups)

    print("ANOVA Results for Margin Across Groups:")
    print("Group Column:", group_col)
    print("Value Column: Margin")
    print("F-statistic:", f_stat)
    print("P-value:", p_val)
    print("Number of groups with sufficient data:", group_count)


ANOVA Results for Margin Across Groups:
Group Column: PostalCode
Value Column: Margin
F-statistic: 0.8701871430635465
P-value: 0.9827599681961054
Number of groups with sufficient data: 496


Despite extremely significant differences in both claim frequency (p ≈ 0.000000) and severity (p = 0.000102) across postal codes, our analysis reveals NO statistically significant differences in profitability (Margin ANOVA: p = 0.983, F = 0.870). This represents a CRITICAL PARADOX that demands immediate strategic reassessment. Our current pricing model is successfully achieving uniform profitability despite massive underlying risk variations - but this may not be optimal for market positioning and growth.

## ✅H0: No significant risk difference between Women and Men

Alternative Hypothesis (H₁):

- H₁: There is a significant risk difference between Women and Men.

(Meaning: male and female customers have different risk levels.)

In [35]:
from statsmodels.stats.proportion import proportions_ztest

# Keep only Male and Female
df_gender = df[df['Gender'].isin(['Female', 'Male'])]

# Count number of claims per gender
claims_by_gender = df_gender.groupby('Gender')['HasClaim'].sum()
print("Number of claims by gender:\n", claims_by_gender)

# Total number of policies per gender
total_by_gender = df_gender['Gender'].value_counts()
print("\nTotal policies by gender:\n", total_by_gender)

# Prepare data for z-test
successes = claims_by_gender.values  # [Female_claims, Male_claims]
nobs = total_by_gender.loc[claims_by_gender.index].values  # [Female_total, Male_total]

# Run z-test for two proportions
stat, p_value = proportions_ztest(count=successes, nobs=nobs)

print("\nZ-test for Claim Frequency (Female vs Male):")
print("Z-statistic:", stat)
print("p-value:", p_value)

if p_value < 0.05:
    print("→ Reject Null Hypothesis: Claim frequency differs between Female and Male")
else:
    print("→ Fail to Reject Null Hypothesis: No significant difference in Claim Frequency")


Number of claims by gender:
 Gender
Female     1
Male      12
Name: HasClaim, dtype: int64

Total policies by gender:
 Gender
Male      4009
Female     116
Name: count, dtype: int64

Z-test for Claim Frequency (Female vs Male):
Z-statistic: 1.0660325049656267
p-value: 0.2864089483921778
→ Fail to Reject Null Hypothesis: No significant difference in Claim Frequency


The gender risk analysis reveals no statistically significant difference in claim frequency between males and females (p = 0.286). However, this finding is compromised by severe sample size limitations, particularly for female policies (only 116 total policies, 1 claim). The analysis lacks statistical power to draw meaningful conclusions about gender risk differences in the South African context.

In [41]:
from scipy.stats import ttest_ind

# Filter to male and female rows only
df_gendered = df[df['Gender'].isin(['Male', 'Female'])].copy()

# Create Margin column
df_gendered['Margin'] = df_gendered['TotalPremium'] - df_gendered['TotalClaims']

# Split
male_margin = df_gendered[df_gendered['Gender'] == 'Male']['Margin']
female_margin = df_gendered[df_gendered['Gender'] == 'Female']['Margin']

# Run t-test
t_stat_margin, p_margin_gender = ttest_ind(male_margin, female_margin, equal_var=False)
print(f"Margin by Gender: t = {t_stat_margin}, p-value = {p_margin_gender}")

Margin by Gender: t = 0.6645082454449376, p-value = 0.5076836294826044


The gender profitability analysis confirms no statistically significant difference in margins between male and female policyholders (t = 0.665, p = 0.508). Combined with our previous finding of no significant claim frequency difference (p = 0.286), this completes a consistent picture: Gender does not appear to be a significant risk or profitability factor in our current portfolio. However, severe sample size limitations require cautious interpretation.